In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

In [ ]:
url_base = "https://www.gasolinamx.com/estado/ciudad-de-mexico-cdmx/"
tipo_combustible = ["Magna", "Premium", "Diesel"]
delegaciones_cdmx = ['alvaro-obregon', 'azcapotzalco', 'benito-juarez', 'coyoacan', 'cuajimalpa-de-morelos', 'cuauhtemoc',
                     'gustavo-a-madero', 'iztacalco', 'iztapalapa', 'la-magdalena-contreras', 'miguel-hidalgo', 'milpa-alta', 
                     'tlahuac', 'tlalpan', 'venustiano-carranza', 'xochimilco']
fecha = datetime.datetime.now().strftime("%d-%m-%Y")
data = []

In [ ]:
def obtener_contneido_pagina(url):
    pagina = requests.get(url)
    return pagina.content

In [ ]:
def genera_soup(html):
    return BeautifulSoup(html, "html.parser") 

In [ ]:
def procesar_pagina(soup, fecha, delegacion):

    tr_magna = soup.find("tr", class_ = "bg-green")
    precio_magna = tr_magna.text[6:].strip()

    tr_premium = soup.find("tr", class_ = "bg-red")
    precio_premium = tr_premium.text[9:].strip()

    tr_diesel = soup.find("tr", class_ = "bg-black")
    precio_diesel = tr_diesel.text[7:].strip()

    precios = [precio_magna, precio_premium, precio_diesel]
    
    for i in range(len(precios)):
        data.append({"Fecha" : fecha,
                     "Delegacion" : delegacion,
                     "Combustible" : tipo_combustible[i],
                     "Precio" : precios[i]})
      
    return data

In [ ]:
def manejar_paginacion(url_base):
    for delegacion in delegaciones_cdmx:
        url = url_base + delegacion
        html = obtener_contneido_pagina(url)
        soup = genera_soup(html)
        procesar_pagina(soup, fecha, delegacion)
    return pd.DataFrame(data)

In [ ]:
df_hoy = manejar_paginacion(url_base)
def actualiza_base(df_hoy):
    df_base = pd.read_csv(".../csv_gasolina/Base.csv")
    df_nuevo = pd.concat([df_base, df_hoy])
    df_nuevo.to_csv("csv_gasolina/" + fecha + " actualizacion.csv")

In [ ]:
# Esto correrlo mañana, antes corroborar que se actualizaron los valores
# df_nuevo = actualiza_base(df_hoy)